# Monthly vs. Hourly Mismatch Evaluation, all results in 1 CSV file

Saves all results into one big excel file!

Fixed tilt with torquetube. 

In [1]:
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.
    
import numpy as np

In [3]:
# The below routine creates a HEXAGONAL torque tube, for a 2-UP configuration of a specific module size. Parameters for the module, the torque tube, and the scene are below.
# This is being run with gendaylit, for one specific timestamp
# MODIFY THE TEST FOLDER TO YOUR LIKING. I utmostly dislike the interactive directory.
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo4'
writefiletitle = r'C:\Users\sayala\Documents\RadianceScenes\Demo4\results\Fixed_Shading_Compiled_April.csv'
timeindex = 4020 # Noon, June 17th. 
simulationname = 'New_Panel'

import csv

# MakeModule Parameters
moduletype=simulationname
numpanels = 1
x = 1  
y = 2
xgap = 0.01
ygap = 0.0
zgap = 0.02
sensorsy = 200 

# TorqueTube Parameters
axisofrotationTorqueTube=False
torqueTube = True
tubetype = 'square'
diameter = 0.20
torqueTubeMaterial = 'black'       # IT's NOT GRAY, IT's GREY
cellLevelModule = False
numcellsx=0
numcellsy=0

# SceneDict Parameters
pitch = 4
albedo = 0.62 
clearance_height = 0.5  
nMods = 20
nRows = 7
azimuth_ang=180
tilt =30

# Now let's run the example

demo = RadianceObj(simulationname,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
metdata = demo.readEPW(epwfile) # read in the EPW weather data from above

# Making module with all the variables
moduleDict=demo.makeModule(name=moduletype,x=x,y=y,bifi=1, 
           torquetube=torqueTube, diameter = diameter, tubetype = tubetype, 
                           material = torqueTubeMaterial, zgap = zgap, numpanels = numpanels, ygap = ygap, 
                           rewriteModulefile = True, xgap=xgap, 
                           axisofrotationTorqueTube=axisofrotationTorqueTube, cellLevelModule=cellLevelModule, 
                           numcellsx=numcellsx, numcellsy = numcellsy)
# create a scene with all the variables
sceneDict = {'tilt':tilt,'pitch': pitch,'height':clearance_height,'azimuth':azimuth_ang, 'module_type':simulationname, 'nMods': nMods, 'nRows': nRows}  
scene = demo.makeScene(moduletype=moduletype, sceneDict=sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.

with open (writefiletitle,'w') as csvfile:

    sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')

    outputheader=['Latitude(deg)','Longitude(deg)', 'Tilt(deg)']
    outputheadervars=[metdata.latitude, metdata.longitude, sceneDict['tilt']]

    if axisofrotationTorqueTube==False:
        outputheader+=['axisofrotationTorqueTube']
        outputheadervars.append(axisofrotationTorqueTube)

    sw.writerow(outputheader)
    sw.writerow(outputheadervars)
    
    outputheadertitles=['Timestamp','Back Avg', 'FrontAvg', 'Ratio']
    
    xheader=[]
    yheader=[]
    zheader=[]
    zrearheader=[]
    mattypeheader=[]
    rearmattypeheader=[]
    frontIrradheader=[]
    backIrradheader=[]
    import string
    
    for i in range (0, sensorsy):
        xheader.append('x_'+str(i))
        yheader.append('y_'+str(i))
        zheader.append('z_'+str(i))
        zrearheader.append('rearZ_'+str(i))
        mattypeheader.append('mattype_'+str(i))
        rearmattypeheader.append('rearMat_'+str(i))
        frontIrradheader.append('Wm2Front_'+str(i))
        backIrradheader.append('Wm2Back_'+str(i))
    
    outputheadertitles+=xheader
    outputheadertitles+=yheader
    outputheadertitles+=zheader
    outputheadertitles+=zrearheader
    outputheadertitles+=mattypeheader
    outputheadertitles+=rearmattypeheader
    outputheadertitles+=frontIrradheader
    outputheadertitles+=backIrradheader


    sw.writerow(outputheadertitles)

    for timeindex in range (2160,3000):  #2160
        demo.gendaylit(metdata,timeindex)  # Noon, June 17th
        octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.
        analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance

        hubheight= (sceneDict['height']+moduleDict['sceney']/2.0*np.sin(sceneDict['tilt']*np.pi/180))

        frontscan, backscan = analysis.moduleAnalysis(hubheight, sceneDict['azimuth'], sceneDict['tilt'], 
                                                  sceneDict['pitch'], sceneDict['nMods'], sceneDict['nRows'], 
                                                  moduleDict['sceney'], moduleDict['scenex'], moduleDict['moduleoffset'],
                                                  sensorsy=sensorsy)

        frontDict, backDict=analysis.analysis(octfile, "Shading_"+str(timeindex), frontscan, backscan)  # compare the back vs front irradiance
        if frontDict is not None:
            print('Annual bifacial ratio average:  %0.3f \n' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )
            outputvalues=[]
            outputvalues.append(timeindex)
            outputvalues.append(sum(analysis.Wm2Back))
            outputvalues.append(sum(analysis.Wm2Front))
            outputvalues.append( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) )
            
            # dictionary
            dictvalues=[]
            dictvalues.append(frontDict['x'])
            dictvalues.append(frontDict['y'])
            dictvalues.append(frontDict['z'])
            dictvalues.append(backDict['z'])
            dictvalues.append(frontDict['mattype'])
            dictvalues.append(backDict['mattype'])
            dictvalues.append(frontDict['Wm2'])
            dictvalues.append(backDict['Wm2'])
            flattened = [val for sublist in dictvalues for val in sublist]
            outputvalues+=flattened
            outputclean = str(outputvalues).translate(string.maketrans('', ''), '[]\'')
            sw.writerow(outputvalues)

print("Finished")


path = C:\Users\sayala\Documents\RadianceScenes\Demo4
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
('\nModule Name:', 'New_Panel')
REWRITING pre-existing module file. 
Module New_Panel successfully created
Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Adjusting solarposition for sunrise hour, 2165 timeindex
Original datetime 1967-04-01 06:00:00-05:00
Localized and adjusted datetime 1967-04-01 05:58:00-05:00 

Missing files, skipping...
Analysis aborted - no octfile 

Created New_Panel.oct
Linescan in process: Shading_2166_Front
Linescan in process: Shading_2166_Back
Saved: results\irr_Shading_2166.csv
Annual bifacial ratio average:  0.260 

Created New_Panel.oct
Linescan in process: Shading_2167_Front
Linescan in pro

Saved: results\irr_Shading_2220.csv
Annual bifacial ratio average:  0.128 

Created New_Panel.oct
Linescan in process: Shading_2221_Front
Linescan in process: Shading_2221_Back
Saved: results\irr_Shading_2221.csv
Annual bifacial ratio average:  0.125 

Created New_Panel.oct
Linescan in process: Shading_2222_Front
Linescan in process: Shading_2222_Back
Saved: results\irr_Shading_2222.csv
Annual bifacial ratio average:  0.128 

Created New_Panel.oct
Linescan in process: Shading_2223_Front
Linescan in process: Shading_2223_Back
Saved: results\irr_Shading_2223.csv
Annual bifacial ratio average:  0.134 

Created New_Panel.oct
Linescan in process: Shading_2224_Front
Linescan in process: Shading_2224_Back
Saved: results\irr_Shading_2224.csv
Annual bifacial ratio average:  0.155 

Created New_Panel.oct
Linescan in process: Shading_2225_Front
Linescan in process: Shading_2225_Back
Saved: results\irr_Shading_2225.csv
Annual bifacial ratio average:  0.230 

Adjusting solarposition for sunset hour

Created New_Panel.oct
Linescan in process: Shading_2286_Front
Linescan in process: Shading_2286_Back
Saved: results\irr_Shading_2286.csv
Annual bifacial ratio average:  0.253 

Created New_Panel.oct
Linescan in process: Shading_2287_Front
Linescan in process: Shading_2287_Back
Saved: results\irr_Shading_2287.csv
Annual bifacial ratio average:  0.244 

Created New_Panel.oct
Linescan in process: Shading_2288_Front
Linescan in process: Shading_2288_Back
Saved: results\irr_Shading_2288.csv
Annual bifacial ratio average:  0.227 

Created New_Panel.oct
Linescan in process: Shading_2289_Front
Linescan in process: Shading_2289_Back
Saved: results\irr_Shading_2289.csv
Annual bifacial ratio average:  0.221 

Created New_Panel.oct
Linescan in process: Shading_2290_Front
Linescan in process: Shading_2290_Back
Saved: results\irr_Shading_2290.csv
Annual bifacial ratio average:  0.222 

Created New_Panel.oct
Linescan in process: Shading_2291_Front
Linescan in process: Shading_2291_Back
Saved: results

Created New_Panel.oct
Linescan in process: Shading_2345_Front
Linescan in process: Shading_2345_Back
Saved: results\irr_Shading_2345.csv
Annual bifacial ratio average:  0.234 

Adjusting solarposition for sunset hour, 2346 timeindex
Original datetime 1967-04-08 19:00:00-05:00
Localized and adjusted datetime 1967-04-08 18:19:00-05:00 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Adjusting sola

Created New_Panel.oct
Linescan in process: Shading_2410_Front
Linescan in process: Shading_2410_Back
Saved: results\irr_Shading_2410.csv
Annual bifacial ratio average:  0.129 

Created New_Panel.oct
Linescan in process: Shading_2411_Front
Linescan in process: Shading_2411_Back
Saved: results\irr_Shading_2411.csv
Annual bifacial ratio average:  0.125 

Created New_Panel.oct
Linescan in process: Shading_2412_Front
Linescan in process: Shading_2412_Back
Saved: results\irr_Shading_2412.csv
Annual bifacial ratio average:  0.123 

Created New_Panel.oct
Linescan in process: Shading_2413_Front
Linescan in process: Shading_2413_Back
Saved: results\irr_Shading_2413.csv
Annual bifacial ratio average:  0.124 

Created New_Panel.oct
Linescan in process: Shading_2414_Front
Linescan in process: Shading_2414_Back
Saved: results\irr_Shading_2414.csv
Annual bifacial ratio average:  0.125 

Created New_Panel.oct
Linescan in process: Shading_2415_Front
Linescan in process: Shading_2415_Back
Saved: results

...

Created New_Panel.oct
Linescan in process: Shading_2718_Front
Linescan in process: Shading_2718_Back
Saved: results\irr_Shading_2718.csv
Annual bifacial ratio average:  0.290 

Created New_Panel.oct
Linescan in process: Shading_2719_Front
Linescan in process: Shading_2719_Back
Saved: results\irr_Shading_2719.csv
Annual bifacial ratio average:  0.225 

Created New_Panel.oct
Linescan in process: Shading_2720_Front
Linescan in process: Shading_2720_Back
Saved: results\irr_Shading_2720.csv
Annual bifacial ratio average:  0.188 

Created New_Panel.oct
Linescan in process: Shading_2721_Front
Linescan in process: Shading_2721_Back
Saved: results\irr_Shading_2721.csv
Annual bifacial ratio average:  0.164 

Created New_Panel.oct
Linescan in process: Shading_2722_Front
Linescan in process: Shading_2722_Back
Saved: results\irr_Shading_2722.csv
Annual bifacial ratio average:  0.151 

Created New_Panel.oct
Linescan in process: Shading_2723_Front
Linescan in process: Shading_2723_Back
Saved: results

Created New_Panel.oct
Linescan in process: Shading_2774_Front
Linescan in process: Shading_2774_Back
Saved: results\irr_Shading_2774.csv
Annual bifacial ratio average:  0.246 

Created New_Panel.oct
Linescan in process: Shading_2775_Front
Linescan in process: Shading_2775_Back
Saved: results\irr_Shading_2775.csv
Annual bifacial ratio average:  0.243 

Created New_Panel.oct
Linescan in process: Shading_2776_Front
Linescan in process: Shading_2776_Back
Saved: results\irr_Shading_2776.csv
Annual bifacial ratio average:  0.250 

Created New_Panel.oct
Linescan in process: Shading_2777_Front
Linescan in process: Shading_2777_Back
Saved: results\irr_Shading_2777.csv
Annual bifacial ratio average:  0.250 

Adjusting solarposition for sunset hour, 2778 timeindex
Original datetime 1967-04-26 19:00:00-05:00
Localized and adjusted datetime 1967-04-26 18:27:00-05:00 

Created New_Panel.oct
Linescan in process: Shading_2778_Front
Linescan in process: Shading_2778_Back
Saved: results\irr_Shading_2778

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Adjusting solarposition for sunrise hour, 2837 timeindex
Original datetime 1967-04-29 06:00:00-05:00
Localized and adjusted datetime 1967-04-29 05:39:00-05:00 

Created New_Panel.oct
Linescan in process: Shading_2837_Front
Linescan in process: Shading_2837_Back
Saved: results\irr_Shading_2837.csv
Annual bifacial ratio average:  0.252 

Created New_Panel.oct
Linescan in process: Shading_2838_Front
Linescan in process: Shading_2838_Back
Saved: results\irr_Shading_2838.csv
Annual bifacial ratio average:  0.330 

Created New_Panel.oct
Linescan in process: Shading_2839_Front
Linescan in process: Shading_2839_Back
Saved: results\irr_Shading_2839.csv
Annual bifacial ratio average:  0.215 

Created New_Panel.oct
Linescan in process: Shading_2840_Front
Linescan in process: Shading_2840_Back
Saved: results\irr_Shading_284

Linescan in process: Shading_2891_Back
Saved: results\irr_Shading_2891.csv
Annual bifacial ratio average:  0.139 

Created New_Panel.oct
Linescan in process: Shading_2892_Front
Linescan in process: Shading_2892_Back
Saved: results\irr_Shading_2892.csv
Annual bifacial ratio average:  0.139 

Created New_Panel.oct
Linescan in process: Shading_2893_Front
Linescan in process: Shading_2893_Back
Saved: results\irr_Shading_2893.csv
Annual bifacial ratio average:  0.140 

Created New_Panel.oct
Linescan in process: Shading_2894_Front
Linescan in process: Shading_2894_Back
Saved: results\irr_Shading_2894.csv
Annual bifacial ratio average:  0.147 

Created New_Panel.oct
Linescan in process: Shading_2895_Front
Linescan in process: Shading_2895_Back
Saved: results\irr_Shading_2895.csv
Annual bifacial ratio average:  0.166 

Created New_Panel.oct
Linescan in process: Shading_2896_Front
Linescan in process: Shading_2896_Back
Saved: results\irr_Shading_2896.csv
Annual bifacial ratio average:  0.213 



Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Adjusting solarposition for sunrise hour, 2957 timeindex
Original datetime 1974-05-04 06:00:00-05:00
Localized and adjusted datetime 1974-05-04 05:36:00-05:00 

Created New_Panel.oct
Linescan in process: Shading_2957_Front
Linescan in process: Shading_2957_Back
Saved: results\irr_Shading_2957.csv
Annual bifacial ratio average:  0.260 

Created New_Panel.oct
Linescan in process: Shading_2958_Front
Linescan in process: Shading_2958_Back
Saved: results\irr_Shading_2958.csv
Annual bifacial ratio average:  0.338 

Created New_Panel.oct
Linescan in process: Shading_2959_Front
Linescan in process: Shading_2959_Back
Saved: results\irr_Shading_295

In [ ]:
# The below routine creates a HEXAGONAL torque tube, for a 2-UP configuration of a specific module size. Parameters for the module, the torque tube, and the scene are below.
# This is being run with gendaylit, for one specific timestamp
# MODIFY THE TEST FOLDER TO YOUR LIKING. I utmostly dislike the interactive directory.
writefiletitle = r'C:\Users\sayala\Documents\RadianceScenes\Demo4\results\Fixed_Shading_Compiled_Aril-June.csv'

with open (writefiletitle,'w') as csvfile:

    sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')

    outputheader=['Latitude(deg)','Longitude(deg)', 'Tilt(deg)']
    outputheadervars=[metdata.latitude, metdata.longitude, sceneDict['tilt']]

    if axisofrotationTorqueTube==False:
        outputheader+=['axisofrotationTorqueTube']
        outputheadervars.append(axisofrotationTorqueTube)

    sw.writerow(outputheader)
    sw.writerow(outputheadervars)
    
    outputheadertitles=['Timestamp','Back Avg', 'FrontAvg', 'Ratio']
    
    xheader=[]
    yheader=[]
    zheader=[]
    zrearheader=[]
    mattypeheader=[]
    rearmattypeheader=[]
    frontIrradheader=[]
    backIrradheader=[]
    import string
    
    for i in range (0, sensorsy):
        xheader.append('x_'+str(i))
        yheader.append('y_'+str(i))
        zheader.append('z_'+str(i))
        zrearheader.append('rearZ_'+str(i))
        mattypeheader.append('mattype_'+str(i))
        rearmattypeheader.append('rearMat_'+str(i))
        frontIrradheader.append('Wm2Front_'+str(i))
        backIrradheader.append('Wm2Back_'+str(i))
    
    outputheadertitles+=xheader
    outputheadertitles+=yheader
    outputheadertitles+=zheader
    outputheadertitles+=zrearheader
    outputheadertitles+=mattypeheader
    outputheadertitles+=rearmattypeheader
    outputheadertitles+=frontIrradheader
    outputheadertitles+=backIrradheader


    sw.writerow(outputheadertitles)

    for timeindex in range (2160,8759):  
        demo.gendaylit(metdata,timeindex)  # Noon, June 17th
        octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.
        analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance

        hubheight= (sceneDict['height']+moduleDict['sceney']/2.0*np.sin(sceneDict['tilt']*np.pi/180))

        frontscan, backscan = analysis.moduleAnalysis(hubheight, sceneDict['azimuth'], sceneDict['tilt'], 
                                                  sceneDict['pitch'], sceneDict['nMods'], sceneDict['nRows'], 
                                                  moduleDict['sceney'], moduleDict['scenex'], moduleDict['moduleoffset'],
                                                  sensorsy=sensorsy)

        frontDict, backDict=analysis.analysis(octfile, "Shading_"+str(timeindex), frontscan, backscan)  # compare the back vs front irradiance
        if frontDict is not None:
            print('Annual bifacial ratio average:  %0.3f \n' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )
            outputvalues=[]
            outputvalues.append(timeindex)
            outputvalues.append(sum(analysis.Wm2Back))
            outputvalues.append(sum(analysis.Wm2Front))
            outputvalues.append( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) )
            
            # dictionary
            dictvalues=[]
            dictvalues.append(frontDict['x'])
            dictvalues.append(frontDict['y'])
            dictvalues.append(frontDict['z'])
            dictvalues.append(backDict['z'])
            dictvalues.append(frontDict['mattype'])
            dictvalues.append(backDict['mattype'])
            dictvalues.append(frontDict['Wm2'])
            dictvalues.append(backDict['Wm2'])
            flattened = [val for sublist in dictvalues for val in sublist]
            outputvalues+=flattened
            outputclean = str(outputvalues).translate(string.maketrans('', ''), '[]\'')
            sw.writerow(outputvalues)

print("Finished")
